In [ ]:
!pip install bitsandbytes
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install datasets

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Ensure you've set the HF_TOKEN environment variable with your Hugging Face token
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
os.environ['HF_TOKEN'] = ''
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_auth_token=os.environ['HF_TOKEN'])

# # If you have a GPU available
# if torch.cuda.is_available():
#     model.cuda()  # This moves your model to the default GPU.


In [ ]:
from datasets import load_dataset

# Load and select the first 100 examples
full_dataset = load_dataset("tasksource/Boardgame-QA")
subset_dataset = full_dataset['train'].select(range(100))  # Adjust based on your dataset's structure

In [ ]:
def preprocess_for_generation(examples):
    # Assuming 'explanation' is a field in your dataset, adjust if it's named differently
    input_texts = examples['example']  # Directly use 'explanation' as input

    # Proofs as targets
    target_texts = examples['proof']

    # Tokenize the input and target texts
    model_inputs = tokenizer(input_texts, max_length=512, padding="max_length", truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target_texts, max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
from datasets import load_dataset
from torch.utils.data import DataLoader
# Split the dataset
split_dataset = subset_dataset.train_test_split(test_size=0.1)  # Adjust test_size as needed
train_dataset1 = split_dataset['train']
test_dataset1 = split_dataset['test']
def tokenize_function(examples):
    # Combine context into input text and proof as target
    input_texts = ["Your input construction logic here" for example in examples['example']]  # Adjust accordingly
    target_texts = examples['proof']  # Assuming proof is the target

    model_inputs = tokenizer(input_texts, max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target_texts, max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
train_dataset = train_dataset1.map(tokenize_function, batched=True)
test_dataset = test_dataset1.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
batch_size = 2  # or whatever batch size is appropriate for your setup

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
from transformers import AdamW
from tqdm.auto import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

model.train()
for epoch in range(3):  # Adjust the number of epochs as needed
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch: {epoch}, Loss: {loss.item()}")

In [ ]:
new_example = "A few players are playing a boardgame. The current state of the game is as follows. The pelikan assassinated the mayor, and is currently in Nigeria. And the rules of the game are as follows. Rule1: One of the rules of the game is that if the pelikan does not neglect the crow, then the crow will, without hesitation, manage to convince the husky. Rule2: Here is an important piece of information about the pelikan: if it is in Africa at the moment then it does not neglect the crow for sure. Rule3: If the pelikan voted for the mayor, then the pelikan does not neglect the crow. Based on the game state and the rules and preferences, does the crow manage to convince the husky?"
input_ids = tokenizer.encode(new_example, return_tensors="pt").to(model.device)

# Generating the proof
generated_ids = model.generate(input_ids, max_length=512)
generated_proof = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(generated_proof)

A few players are playing a boardgame. The current state of the game is as follows. The pelikan assassinated the mayor, and is currently in Nigeria. And the rules of the game are as follows. Rule1: One of the rules of the game is that if the pelikan does not neglect the crow, then the crow will, without hesitation, manage to convince the husky. Rule2: Here is an important piece of information about the pelikan: if it is in Africa at the moment then it does not neglect the crow for sure. Rule3: If the pelikan voted for the mayor, then the pelikan does not neglect the crow. Based on the game state and the rules and preferences, does the crow manage to convince the husky? maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maroc maro

In [ ]:
# Placeholder for storing example and generated proof pairs
generated_proofs = []

# Assuming 'test_dataset' is already prepared and tokenized
for example in test_dataset1:
    # Prepare the input for the model
    input_ids = tokenizer.encode(example['example'], return_tensors="pt").to(model.device)

    # Generate proof
    generated_ids = model.generate(input_ids, max_length=1000)
    generated_proof = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # Store the results
    generated_proofs.append({"example": example['example'], "generated_proof": generated_proof,"label": example['label']})

In [ ]:
import pandas as pd

# Convert the results to a DataFrame
df = pd.DataFrame(generated_proofs)

# Save the DataFrame to a CSV file
df.to_csv("generated_proofs_ft.csv", index=False)